# RAPL sysfs Interface

**Note:** this notebook is meant to be executed on the supplied Docker image.

**Note:** this and all other notebooks use the sysfs interface of the system where the Jupyter Notebook is running. It means that the output you see may be different on different systems, and some systems may not support Intel RAPL at all.

In [ ]:
import os
from pathlib import Path

First, we have to check whether RAPL sysfs interface is present which is signified by the existence of the directory `/sys/devices/virtual/powercap/intel-rapl`, which contains all files related to the RAPL sysfs interface.

In [ ]:
if os.path.isdir('/sys/devices/virtual/powercap/intel-rapl'):
    print('RAPL sysfs supported')
else:
    print('RAPL sysfs unsupported')

In order to list all supported domains, we can list the direcotry `/sys/class/powercap`, which contains a flat structure of zones (every zone is represented by a directory in `/sys/class/powercap`), compared to the hierarchical structure from `/sys/devices/virtual/powercap` (directories representing zones contain directories representing subzones).

In [ ]:
domains = os.listdir('/sys/class/powercap')

for domain in domains:
    print(f'Domain {domain}:')
    try:
        name = Path(f'/sys/class/powercap/{domain}/name').read_text().strip()
        print(f'  name: {name}')
    except FileNotFoundError:
        print(f'  name: unknown')
    try:
        enabled = Path(f'/sys/class/powercap/{domain}/enabled').read_text().strip()
        print(f'  controls enabled: {enabled}')
    except FileNotFoundError:
        print(f'  controls enabled: unknown')
    print('')

Let's list all supported CPUs. They are represented by domains in the form of `intel-rapl:<number>`, and their name is in the form of `package-<number>`.

In [ ]:
files = os.listdir('/sys/devices/virtual/powercap/intel-rapl')
first_cpu_domain = None
cpu_count = 0

for file in files:
    if file.startswith('intel-rapl:'):
        name = Path(f'/sys/devices/virtual/powercap/intel-rapl/{file}/name').read_text().strip()
        if name.startswith('package-'):
            if first_cpu_domain is None: first_cpu_domain = file
            cpu_count += 1

            print(f'CPU "{name}" (zone {file})')

print('')
print(f'There are {cpu_count} CPUs in total')

Let's take the first CPU and list all subzones within this CPU.

In [ ]:
files = os.listdir(f'/sys/devices/virtual/powercap/intel-rapl/{first_cpu_domain}')
subzone_count = 0

for file in files:
    if file.startswith('intel-rapl:'):
        subzone_count += 1
        name = Path(f'/sys/devices/virtual/powercap/intel-rapl/{first_cpu_domain}/{file}/name').read_text().strip()
        print(f'Subzone {file} with name "{name}"')

print('')
print(f'There are {subzone_count} subzones in total')

The code below prints the file structures of `/sys/class/powercap` and `/sys/devices/virtual/powercap`.

In [ ]:
def print_tree(directory, indent=''):
    files = os.listdir(directory)
    for file in files:
        path = f'{directory}/{file}'
        if os.path.islink(path):
            print(f'{indent}{file} -> {os.readlink(path)}')
            continue
        print(f'{indent}{file}')
        if os.path.isdir(path):
            print_tree(path, f'{indent}  ')

print('/sys/class/powercap')
print_tree('/sys/class/powercap', '  ')
print('')
print('/sys/devices/virtual/powercap')
print_tree('/sys/devices/virtual/powercap', '  ')